In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchsummary import summary
from models.KANFormer import KANFormer
from utils import get_model_size

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# (self,HSI_bands=31,MSI_bands=3,hidden_dim=256,scale=4,depth=4,image_size=64)
myKAN = KANFormer(HSI_bands=31,MSI_bands=3,hidden_dim=256,scale=2,depth=4,image_size=64)

In [8]:
myKAN = myKAN.to(device)

In [10]:
model_dir = './trained_models/CAVEx2_epoch:999,PSNR:54.62.pth'
#model_dir = './trained_models/CAVEx4_epoch:999,PSNR:49.17.pth'
checkpoint = torch.load(model_dir)
myKAN.load_state_dict(checkpoint['net'], strict=False)
# Set for forward use
myKAN.eval()

/tmp/ipykernel_14409/3270711370.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_dir)


KANFormer(
  (fusion): Fusion(
    (hsi_kan): KANLinear(
      (base_activation): SiLU()
    )
    (msi_kan): KANLinear(
      (base_activation): SiLU()
    )
    (align_kan): KANLinear(
      (base_activation): SiLU()
    )
  )
  (layers): ModuleList(
    (0-3): 4 x KANBlock(
      (kan_layer_01): KANLinear(
        (base_activation): SiLU()
      )
      (kan_layer_02): KANLinear(
        (base_activation): SiLU()
      )
    )
  )
  (refine): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 31, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

In [16]:
#from torchinfo import summary
from torchsummary import summary
summary(myKAN,input_size=[(16,16,31),(64,64,3)])

AssertionError: 

In [72]:
model = KANFormer(scale=4,depth=4)
model = model.to(device)
checkpoint = torch.load(model_dir)
model.load_state_dict(checkpoint['net'],strict=False)


/tmp/ipykernel_401/318078183.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_dir)


RuntimeError: Error(s) in loading state_dict for KANFormer:
	size mismatch for layers.0.kan_layer_01.spline_weight: copying a param with shape torch.Size([256, 256, 8]) from checkpoint, the shape in current model is torch.Size([256, 256, 261]).
	size mismatch for layers.0.kan_layer_01.grid: copying a param with shape torch.Size([256, 12]) from checkpoint, the shape in current model is torch.Size([256, 518]).
	size mismatch for layers.0.kan_layer_02.spline_weight: copying a param with shape torch.Size([256, 256, 8]) from checkpoint, the shape in current model is torch.Size([256, 256, 261]).
	size mismatch for layers.0.kan_layer_02.grid: copying a param with shape torch.Size([256, 12]) from checkpoint, the shape in current model is torch.Size([256, 518]).
	size mismatch for layers.1.kan_layer_01.spline_weight: copying a param with shape torch.Size([256, 256, 8]) from checkpoint, the shape in current model is torch.Size([256, 256, 261]).
	size mismatch for layers.1.kan_layer_01.grid: copy

In [69]:
print(model['net']['fusion.hsi_kan.spline_weight'].shape)
print(model['net']['fusion.hsi_kan.base_weight'].shape)
print(model['net']['layers.0.kan_layer_01.spline_weight'].shape)

TypeError: 'KANFormer' object is not subscriptable

In [ ]:
N = np.ones((5,4))
print(N.shape)
print(N.reshape((-1,)+N.shape).shape)

In [ ]:
(1,)+(2,3)

In [ ]:
N

In [ ]:
summary(model, input_size=[(31,16,16),(3,64,64)])

# Fuse mock data


In [ ]:
gt = np.random.binomial(100,0.7,size=((64,64,31)))
plt.matshow(gt[:,:,0])

In [ ]:
from skimage.transform import resize
from skimage.measure import block_reduce

lr_hs = resize(gt, (16, 16, 31), anti_aliasing=True)
plt.matshow(lr_hs[:,:,0])
